In [1]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim import models
from gensim.models import LdaModel, CoherenceModel
from gensim.models.wrappers import LdaMallet
import pprint

# spacy for lemmatization
import spacy
from spacy.lang.en import English

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np

reviews_datasets = pd.read_csv("F:\\BUET\Project\\Code\\Dataset\\ds_05032021_test.csv",encoding="latin-1")
#reviews_datasets = reviews_datasets.head(20)
reviews_datasets.dropna()
obody = reviews_datasets['Body']
cview= reviews_datasets['ViewCount']
reviews_datasets.head()
#reviews_datasets.to_csv('D:\\test.csv')



,Id,AnswerCount,Body,CommentCount,CreationDate,FavoriteCount,PostTypeId,Score,Tags,Title,ViewCount
0,56364909,0,<p>We are creating pvc using yml file. The pv...,2,May 29 2019 4:26PM,0,1,0,<kubernetes><ansible>,How to validate all the pvc created by yml usi...,26
1,56367775,2,"<p>I am new to using Ansible-Playbooks, and I ...",0,May 29 2019 8:06PM,0,1,0,<ansible><splunk>,Accepting Splunk license agreement using Ansib...,677
2,56367942,1,<p>How to run Ansible playbooks and commands f...,0,May 29 2019 8:20PM,0,1,0,<python><ansible>,Calling Ansible adhoc commands/playbooks in py...,305
3,56368589,1,<p>I have the following ansible playbooks:</p>...,2,May 29 2019 9:12PM,0,1,-1,<ansible>,"using the unix command ""source"" in ansible",209
4,56369126,1,<p>I'm trying to move all files under a specif...,0,May 29 2019 10:03PM,0,1,0,<ansible><remote-server><ansible-2.x>,Using Ansible copy module for remote to remote...,414


In [4]:
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk

import re
from nltk.corpus import stopwords
from nltk.tokenize import ToktokTokenizer
from nltk.stem.snowball import SnowballStemmer

tokenizer = ToktokTokenizer()
stemmer = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))

# Preprocess the text for vectorization
# - Remove HTML
# - Remove stopwords
# - Remove special characters
# - Convert to lowercase
# - Stemming

In [5]:
import re

# Convert to list
data = reviews_datasets.Body.values.tolist()

# Remove new line characters
data = [re.sub(r'\s+', ' ', sent) for sent in data]

# Remove distracting single and double quotes
data = [re.sub("\'", "", sent) for sent in data]
data = [re.sub('\", "', '', sent) for sent in data]
data = [re.sub('\\"', '', sent) for sent in data]
data = [re.sub('\"', '', sent) for sent in data]
data = [re.sub('[\\:"]', '', sent) for sent in data]

# Remove web links
data = [re.sub(r'^https?:\/\/.*[\r\n]*', '', sent) for sent in data]

print(data[:3])

['<p>We are creating pvc using yml file. The pvc entries are created dynamically based on user input. No of pvc to be created will be received as input and accordingly the number of pvc entries will be created in yml file then that yml file will be passed to Kubectl command to create pvc.</p> <pre><code>--- kind PersistentVolumeClaim apiVersion v1 metadata name pvc{{ currenttimestamp }} labels ansible {{ currenttimestamp }} spec accessModes - ReadWriteOnce resources requests storage {{ local_volume_size | default(10) }}Gi storageClassName sc{{ currenttimestamp }} --- kind PersistentVolumeClaim apiVersion v1 metadata name pvc{{ timestamp }} labels ansible {{ currenttimestamp }} spec accessModes - ReadWriteOnce resources requests storage {{ local_volume_size | default(10) }}Gi storageClassName sc{{ currenttimestamp }} </code></pre> <p>For single PVC validation , i am using below Ansible code</p> <pre><code>- name Verify whether the PVC is created command kubectl get pvc {{pvcname}} -o js

In [6]:
# Tokenize words and text clean up
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:3])

[['we', 'are', 'creating', 'pvc', 'using', 'yml', 'file', 'the', 'pvc', 'entries', 'are', 'created', 'dynamically', 'based', 'on', 'user', 'input', 'no', 'of', 'pvc', 'to', 'be', 'created', 'will', 'be', 'received', 'as', 'input', 'and', 'accordingly', 'the', 'number', 'of', 'pvc', 'entries', 'will', 'be', 'created', 'in', 'yml', 'file', 'then', 'that', 'yml', 'file', 'will', 'be', 'passed', 'to', 'kubectl', 'command', 'to', 'create', 'pvc', 'pre', 'code', 'kind', 'apiversion', 'metadata', 'name', 'pvc', 'labels', 'ansible', 'spec', 'accessmodes', 'readwriteonce', 'resources', 'requests', 'storage', 'default', 'gi', 'sc', 'kind', 'apiversion', 'metadata', 'name', 'pvc', 'timestamp', 'labels', 'ansible', 'spec', 'accessmodes', 'readwriteonce', 'resources', 'requests', 'storage', 'default', 'gi', 'sc', 'code', 'pre', 'for', 'single', 'pvc', 'validation', 'am', 'using', 'below', 'ansible', 'code', 'pre', 'code', 'name', 'verify', 'whether', 'the', 'pvc', 'is', 'created', 'command', 'kubec

In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=75) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=75)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[3]]])

['have', 'the', 'following', 'ansible', 'playbooks', 'runwithdotslash', 'yml', 'pre', 'code', 'hosts', 'user', 'myhost', 'local', 'tasks', 'name', 'try', 'running', 'script', 'with', 'dot', 'slash', 'command', 'script', 'sh', 'code', 'pre', 'runwithsource', 'yml', 'pre', 'code', 'hosts', 'user', 'myhost', 'local', 'tasks', 'name', 'try', 'running', 'script', 'with', 'source', 'command', 'source', 'script', 'sh', 'code', 'pre', 'when', 'ssh', 'into', 'user', 'myhost', 'local', 'am', 'taken', 'to', 'the', 'home', 'directory', 'of', 'user', 'and', 'can', 'run', 'script', 'sh', 'both', 'with', 'dot', 'slash', 'and', 'with', 'source', 'however', 'only', 'the', 'first', 'playbook', 'works', 'am', 'running', 'the', 'playbooks', 'with', 'the', 'following', 'command', 'pre', 'code', 'ansible', 'playbook', 'runwithdotslash', 'yml', 'ansible', 'playbook', 'runwithsource', 'yml', 'code', 'pre', 'the', 'second', 'gives', 'the', 'following', 'error', 'message', 'pre', 'code', 'failed', 'gt', 'change

In [10]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [11]:
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
#spacy.load('en_core_web_sm')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

#reviews_datasets['topics']=data_lemmatized
#reviews_datasets['obody']=obody
#reviews_datasets.head(20)
#reset option to default value


#reviews_datasets.to_csv("D:\\output1.csv")

[['create', 'pvc', 'use', 'yml', 'file', 'pvc', 'entry', 'create', 'dynamically', 'base', 'user', 'input', 'pvc', 'create', 'receive', 'input', 'accordingly', 'number', 'pvc', 'entry', 'create', 'yml', 'file', 'yml', 'file', 'pass', 'kubectl', 'command', 'create', 'pvc', 'pre', 'code', 'kind', 'apiversion', 'metadata', 'name', 'pvc', 'label', 'ansible', 'spec', 'accessmode', 'readwriteonce', 'resource', 'request', 'storage', 'default', 'gi', 'kind', 'apiversion', 'metadata', 'name', 'pvc', 'timestamp', 'label', 'ansible', 'spec', 'accessmode', 'readwriteonce', 'resource', 'request', 'storage', 'default', 'sc', 'code', 'pre', 'single', 'pvc', 'validation', 'use', 'ansible', 'code', 'pre', 'code', 'name', 'verify', 'pvc', 'create', 'command', 'kubectl', 'get', 'pvc', 'pvcname', 'json', 'register', 'pvcresult', 'code', 'pre', 'pvc', 'generation', 'yml', 'dynamic', 'need', 'get', 'pvc', 'name', 'yml', 'validate', 'simple', 'way', 'check', 'pvc', 'mention', 'yml', 'create'], ['new', 'use', 

In [12]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 2), (1, 1), (2, 3), (3, 2), (4, 1), (5, 1), (6, 5), (7, 2), (8, 7), (9, 2), (10, 1), (11, 1), (12, 2), (13, 3), (14, 1), (15, 2), (16, 1), (17, 2), (18, 1), (19, 2), (20, 2), (21, 2), (22, 1), (23, 2), (24, 4), (25, 1), (26, 1), (27, 1), (28, 4), (29, 13), (30, 1), (31, 1), (32, 2), (33, 1), (34, 1), (35, 2), (36, 2), (37, 1), (38, 1), (39, 1), (40, 2), (41, 2), (42, 1), (43, 2), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 6)]]


In [13]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                            eval_every=10,
                                           per_word_topics=True)

In [21]:
# Print the Keyword in the 15 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.049*"ansible" + 0.022*"playbook" + 0.021*"blockquote" + 0.013*"run" + 0.012*"testworkflow" + 0.012*"api" + 0.012*"job" + 0.012*"d" + 0.012*"handler" + 0.010*"html"'), (1, '0.066*"code" + 0.044*"pre" + 0.042*"host" + 0.028*"gt" + 0.026*"name" + 0.022*"list" + 0.022*"output" + 0.022*"item" + 0.018*"get" + 0.017*"ansible"'), (2, '0.058*"code" + 0.047*"ansible" + 0.024*"pre" + 0.020*"use" + 0.017*"li" + 0.014*"playbook" + 0.013*"server" + 0.011*"host" + 0.010*"try" + 0.009*"python"'), (3, '0.024*"image" + 0.021*"start" + 0.017*"ami" + 0.013*"various" + 0.010*"ec" + 0.010*"build" + 0.010*"vagrant" + 0.009*"tar" + 0.008*"other" + 0.008*"format"'), (4, '0.029*"log" + 0.024*"ansible" + 0.024*"code" + 0.022*"string" + 0.018*"new" + 0.018*"terraform" + 0.016*"key" + 0.015*"install" + 0.014*"pre" + 0.013*"name"'), (5, '0.051*"true" + 0.035*"null" + 0.035*"result" + 0.030*"code" + 0.026*"pre" + 0.026*"fail" + 0.025*"item" + 0.018*"falconctl" + 0.017*"false" + 0.014*"ncat"'), (6, '0.051*"gr

In [22]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.7205923814102215

Coherence Score:  0.39868242811989313


In [14]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
#mallet_path = 'D:/mallet-2.0.8/bin/mallet' # update this path
#lda_mallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=15, id2word=id2word, optimize_interval=10, iterations=1000)

import os
from gensim.models.wrappers import LdaMallet

os.environ['MALLET_HOME'] = 'F:\\mallet-2.0.8'

mallet_path = 'F:\\mallet-2.0.8\\bin\\mallet'
#ldamallet_test = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=dictionary_test)
lda_mallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=15, id2word=id2word, optimize_interval=10, iterations=1000)

In [15]:
# Show Topics
print(lda_mallet.show_topics(num_topics=-1, num_words=20, formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=lda_mallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(0, [('window', 0.02774813233724653), ('user', 0.026680896478121666), ('root', 0.02454642475987193), ('become_user', 0.021344717182497332), ('pulseaudio', 0.021344717182497332), ('shell', 0.0192102454642476), ('register', 0.0192102454642476), ('fail', 0.01814300960512273), ('task', 0.017075773745997867), ('restart', 0.014941302027748132), ('password', 0.013874066168623266), ('myuser', 0.011739594450373533), ('true', 0.010672358591248666), ('eq_exit', 0.010672358591248666), ('machine', 0.010672358591248666), ('install', 0.010672358591248666), ('temp_asdtech', 0.0096051227321238), ('code', 0.0096051227321238), ('linux', 0.008537886872998933), ('powershell', 0.008537886872998933)]), (1, [('group', 0.0738255033557047), ('script', 0.02953020134228188), ('output', 0.028187919463087248), ('sid', 0.01879194630872483), ('param', 0.0174496644295302), ('li', 0.0174496644295302), ('hname', 0.01476510067114094), ('jenkin', 0.013422818791946308), ('item', 0.013422818791946308), ('url', 0.0134228187

In [43]:
def format_topics_sentences(ldamodel=lda_mallet, corpus=corpus, texts=data_lemmatized):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_mallet, corpus=corpus, texts=obody)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head()
#print(df_dominant_topic)
# Show
#df_dominant_topic.head(10)
df_dominant_topic.to_csv("D:\\output_topics_keyword_Including_body_060321.csv")

In [41]:
# Python Topics Modeling
python_topics_modeling = df_dominant_topic.groupby(['Keywords','Dominant_Topic'])['Dominant_Topic'].mean().astype(int)
print(python_topics_modeling)



# Topics Wise average (View)
averag_view_wise_topics = average_wise_topcs = df_dominant_topic.groupby('Keywords')['Text'].mean().astype(int)
#print(averag_view_wise_topics)

#average_wise_topcs.sort_index(ascending =False)

# Topics Wise sum (View)
#df_dominant_topic.groupby('Keywords')['Text'].sum()



Keywords                                                                                  Dominant_Topic
ansible, cfg, home, ssh, host, connection, plugin, user, usr, file                        2.0                2
ansible, li, server, module, file, doc, late, test, api, html                             13.0              13
ansible, python, package, terraform, inventory, install, loader, plugin, ubuntu, option   5.0                5
code, pre, item, result, list, gt, loop, msg, output, debug                               7.0                7
code, pre, playbook, host, ansible, file, task, run, role, error                          3.0                3
code, pre, user, _, create, airflow, follow, command, group, index                        11.0              11
file, copy, tmp, usr, root, local, opt, strong, remote, path                              4.0                4
group, script, output, sid, param, li, hname, item, url, jenkin                           1.0                1
host, d

In [39]:
averag_view_wise_topics.to_csv("D:\\view_wise_topics_modeling.csv")